<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/0621/0621_06_3_KorNLI_%EB%8B%A4%EC%A4%91_%ED%81%B4%EB%9E%98%EC%8A%A4_%EB%B6%84%EB%A5%98(%EC%8B%A4%EC%8A%B5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KorNLI 분류

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
!pip install torch

In [ ]:
import os

import numpy as np
import pandas as pd

from datetime import datetime
import json
import re

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## 데이터셋 로드 및 구조 확인

- 데이터셋 정보 : https://huggingface.co/datasets/klue

In [ ]:
from datasets import load_dataset

cs = load_dataset("klue", "nli", split="train")
cs = cs.train_test_split(0.1)
test_cs = load_dataset("klue", "nli", split="validation")
train_cs = cs["train"]
valid_cs = cs["test"]

Generating train split:   0%|          | 0/24998 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to C:/Users/jj/.cache/huggingface/datasets/klue/nli/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


Found cached dataset klue (C:/Users/jj/.cache/huggingface/datasets/klue/nli/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


In [ ]:
train_cs

Dataset({
    features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
    num_rows: 22498
})

In [ ]:
valid_cs

Dataset({
    features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
    num_rows: 2500
})

In [ ]:
test_cs

Dataset({
    features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
    num_rows: 3000
})

## 데이터셋 전처리

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

훈련 데이터, 검증 데이터, 테스트 데이터에 대해서 `[CLS] 문장 [SEP]` 구조를 만듭니다. [CLS]는 분류를 하기 위해 BERT가 사용하는 첫번째 입력 토큰이며, [SEP]는 입력 문장의 종료를 나타내기 위해 사용하는 스페셜 토큰입니다.

In [ ]:
### 이 코드는 문장 구성을 보여드리기 위한 단순 예시 코드입니다!

# 훈련 데이터, 검증 데이터, 테스트 데이터에 대해서 `[CLS] 문장 [SEP]` 구조를 만들어야 합니다.
train_sentences = list(map(lambda train_cs: '[CLS] ' + str(train_cs['premise']) + ' [SEP] ' + str(train_cs['hypothesis']) + ' [SEP]', train_cs))
validation_sentences = list(map(lambda valid_cs: '[CLS] ' + str(valid_cs['premise']) + ' [SEP] ' + str(valid_cs['hypothesis']) + ' [SEP]', valid_cs))
test_sentences = list(map(lambda test_cs: '[CLS] ' + str(test_cs['premise']) + ' [SEP] ' + str(test_cs['hypothesis']) + ' [SEP]', test_cs))

In [ ]:
test_sentences[:5]

['[CLS] 흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다. [SEP] 어떤 방에서도 흡연은 금지됩니다. [SEP]',
 '[CLS] 10명이 함께 사용하기 불편함없이 만족했다. [SEP] 10명이 함께 사용하기 불편함이 많았다. [SEP]',
 '[CLS] 10명이 함께 사용하기 불편함없이 만족했다. [SEP] 성인 10명이 함께 사용하기 불편함없이 없었다. [SEP]',
 '[CLS] 10명이 함께 사용하기 불편함없이 만족했다. [SEP] 10명이 함께 사용하기에 만족스러웠다. [SEP]',
 '[CLS] 10층에 건물사람들만 이용하는 수영장과 썬베드들이 있구요. [SEP] 건물사람들은 수영장과 썬베드를 이용할 수 있습니다. [SEP]']

In [ ]:
train_labels = train_cs['label']
validation_labels = valid_cs['label']
test_labels = test_cs['label']

얽힘(entailment) : 0
중립(neutral) : 1
모순(contradiction) : 2

## BERT 토크나이저를 이용한 전처리

BERT를 사용하기 위해서는 토크나이저와 모델이 반드시 맵핑 관계여야만 합니다. 다시 말해 아래의 이름에 들어가는 모델이름은 반드시 동일해야 합니다.

* BertTokenizer.from_pretrained('모델이름')
* BertForSequenceClassification.from_pretrained("모델이름")

토크나이저는 내부적으로 Vocabulary를 갖고 있어 정수 인코딩을 수행해주는 모듈입니다.

In [ ]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained("klue/bert-base")

C:\Users\jj\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jj\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
from transformers import BertTokenizer
# 한국어 BERT 중 하나인 'klue/bert-base'를 사용.
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [ ]:
X_train = tokenizer(train_cs['premise'],train_cs['hypothesis'], truncation=True, padding=True, return_tensors = 'tf')
X_valid = tokenizer(valid_cs['premise'],valid_cs['hypothesis'], truncation=True, padding=True, return_tensors = 'tf')
X_test = tokenizer(test_cs['premise'],test_cs['hypothesis'], truncation=True, padding=True, return_tensors = 'tf')

In [ ]:
a, b = train_cs['premise'][0], train_cs['hypothesis'][0]
a, b

('집합투자기구의 모든 소득을 과세대상에 포함해 투자자가 인식하는 실제 소득과 과세소득을 일치시켜 집합투자기구의 손실과세 요인을 해소한다.',
 '집합투자기구의 손실과세 요인은 여러 가지가 있다.')

In [ ]:
emb = tokenizer(a, b, truncation=True, padding=True, return_tensors = 'tf')
emb

{'input_ids': <tf.Tensor: shape=(1, 57), dtype=int32, numpy=
array([[    2, 12958, 13344, 11181,  2079,  3721,  4429,  2069,  7148,
         2104,  2290,  2170,  3954,  2097,  5123,  2116,  4263,  2205,
         2259,  3966,  4429,  2145,  7148,  7625,  2069,  6708,  2067,
         2621, 12958, 13344, 11181,  2079,  5817,  2145,  2103,  5090,
         2069,  5343,  4538,    18,     3, 12958, 13344, 11181,  2079,
         5817,  2145,  2103,  5090,  2073,  3789,  3663,  2116,  1513,
         2062,    18,     3]])>, 'token_type_ids': <tf.Tensor: shape=(1, 57), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])>, 'attention_mask': <tf.Tensor: shape=(1, 57), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
   

In [ ]:
tokenizer.decode(emb['input_ids'])

'[CLS] 집합투자기구의 모든 소득을 과세대상에 포함해 투자자가 인식하는 실제 소득과 과세소득을 일치시켜 집합투자기구의 손실과세 요인을 해소한다. [SEP] 집합투자기구의 손실과세 요인은 여러 가지가 있다. [SEP]'

In [ ]:
X_train

{'input_ids': <tf.Tensor: shape=(22498, 100), dtype=int32, numpy=
array([[    2,  4318,  2522, ...,     0,     0,     0],
       [    2,  1504,  2259, ...,     0,     0,     0],
       [    2, 10989,  2043, ...,     0,     0,     0],
       ...,
       [    2,   918,  3674, ...,     0,     0,     0],
       [    2, 14195, 19521, ...,     0,     0,     0],
       [    2,  7131,  2470, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(22498, 100), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(22498, 100), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype

In [ ]:
import tensorflow as tf

# 주어진 데이터소스를 여러 Tensor로 자른 후 iterator(반복가능 객체)로 만들기
train_dataset = tf.data.Dataset.from_tensor_slices((dict(X_train), train_labels))

val_dataset = tf.data.Dataset.from_tensor_slices((dict(X_valid), validation_labels))

test_dataset = tf.data.Dataset.from_tensor_slices((dict(X_test), test_labels))

훈련 데이터, 검증 데이터, 텍스트 데이터에 대해서 data_to_tensor 함수를 통해서   정수 인코딩 된 데이터, 레이블, 어텐션 마스크를 얻습니다.

In [ ]:
list(train_dataset.take(1))

[({'input_ids': <tf.Tensor: shape=(100,), dtype=int32, numpy=
   array([    2, 12958, 13344, 11181,  2079,  3721,  4429,  2069,  7148,
           2104,  2290,  2170,  3954,  2097,  5123,  2116,  4263,  2205,
           2259,  3966,  4429,  2145,  7148,  7625,  2069,  6708,  2067,
           2621, 12958, 13344, 11181,  2079,  5817,  2145,  2103,  5090,
           2069,  5343,  4538,    18,     3, 12958, 13344, 11181,  2079,
           5817,  2145,  2103,  5090,  2073,  3789,  3663,  2116,  1513,
           2062,    18,     3,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0])>,
   'token_type_ids': <tf.Tensor: shape=(100,), dtype=int32, numpy=
   array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
list(train_dataset.take(1))

[({'input_ids': <tf.Tensor: shape=(99,), dtype=int32, numpy=
   array([    2, 12958, 13344, 11181,  2079,  3721,  4429,  2069,  7148,
           2104,  2290,  2170,  3954,  2097,  5123,  2116,  4263,  2205,
           2259,  3966,  4429,  2145,  7148,  7625,  2069,  6708,  2067,
           2621, 12958, 13344, 11181,  2079,  5817,  2145,  2103,  5090,
           2069,  5343,  4538,    18, 12958, 13344, 11181,  2079,  5817,
           2145,  2103,  5090,  2073,  3789,  3663,  2116,  1513,  2062,
             18,     3,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0])>,
   'token_type_ids': <tf.Tensor: shape=(99,), dtype=int32, numpy=
   array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
print('훈련 데이터의 크기:', len(train_labels))
print('검증 데이터의 크기:', len(validation_labels))
print('테스트 데이터의 크기:', len(test_labels))

훈련 데이터의 크기: 22498
검증 데이터의 크기: 2500
테스트 데이터의 크기: 3000


## GPU가 정상 셋팅되었는지 확인.
Colab에서 GPU를 사용하기 위해서는 아래와 같이 설정이 되어있어야만 합니다.

* 런타임 > 런타임 유형 변경 > 하드웨어 가속기 > 'GPU' 선택

## 모델 로드하기

BERT를 사용하여 텍스트를 분류하는 BERT 아키텍처는 `BertForSequenceClassification.from_pretrained("모델이름")`을 넣어서 가능합니다. 레이블 수로 num_labels라는 인자값에 레이블의 수를 기재해줍니다.

In [ ]:
from transformers import TFBertModel

class MyModel(tf.keras.Model):
    def __init__(self, model_name):
        super().__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt = True)
        self.classifier = tf.keras.layers.Dense(3,
                                                kernel_initializer = tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name = 'classifier')

    def call(self, input_ids = None, token_type_ids = None, attention_mask = None):
        outputs = self.bert(input_ids = input_ids, token_type_ids =token_type_ids, attention_mask=attention_mask)
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)

        return prediction


## 모델 학습

In [ ]:
model = MyModel("klue/bert-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['bert.encoder.layer.5.attention.self.key.weight', 'bert.encoder.layer.7.intermediate.dense.weight', 'bert.encoder.layer.2.attention.self.key.weight', 'bert.encoder.layer.7.attention.self.value.weight', 'bert.encoder.layer.7.attention.output.LayerNorm.weight', 'bert.encoder.layer.8.output.dense.bias', 'bert.encoder.layer.11.attention.self.query.weight', 'bert.encoder.layer.11.intermediate.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'bert.encoder.layer.9.output.LayerNorm.bias', 'bert.encoder.layer.1.attention.self.value.bias', 'bert.pooler.dense.bias', 'bert.encoder.layer.3.attention.output.dense.bias', 'bert.encoder.layer.10.attention.output.LayerNorm.weight', 'bert.encoder.layer.10.attention.self.value.weight', 'bert.encoder.layer.6.output.LayerNorm.weight', 'bert.encoder.layer.9.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

In [ ]:
num_train = 20000
num_test = 2500

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

callback_earlystop = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.001,
    patience=2)

model.fit(
    train_dataset.shuffle(num_train).batch(32),
    epochs=5, batch_size=32,
    validation_data = val_dataset.shuffle(num_test).batch(32),
    callbacks = [callback_earlystop]
)

Epoch 1/5
704/704 [==============================] - 243s 319ms/step - loss: 1.1276 - accuracy: 0.3352 - val_loss: 1.0946 - val_accuracy: 0.4108
Epoch 2/5
704/704 [==============================] - 221s 314ms/step - loss: 1.1014 - accuracy: 0.3577 - val_loss: 1.0593 - val_accuracy: 0.4564
Epoch 3/5
704/704 [==============================] - 221s 314ms/step - loss: 1.0711 - accuracy: 0.4161 - val_loss: 1.0466 - val_accuracy: 0.4540
Epoch 4/5
669/704 [===========================>..] - ETA: 10s - loss: 1.0727 - accuracy: 0.4268

# 예측

In [ ]:
text = {"text" : "흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다.", "text_pair" : "어떤 방에서도 흡연은 금지됩니다."}

# 분류 결과 출력
inputs = tokenizer(text, return_tensors="tf")
model(**inputs)

{'input_ids': <tf.Tensor: shape=(1, 30), dtype=int32, numpy=
array([[    2, 25313,  2377,  2031,  2073, 20812,  2116,  1513,  2259,
         1129, 24094, 20812, 27135,  9753,  2052,  3662, 11800,    18,
            3,  3711,  1129, 27135,  2119,  9753,  2073,  5040,  3598,
         3606,    18,     3]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 30), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 30), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [ ]:
sent1 = "저는, 그냥 알아내려고 거기 있었어요."
sent2 = "나는 돈이 어디로 갔는지 이해하려고 했어요."



2

In [ ]:
sent1 = "저는 그것을 이해하려고 거기 있었어요."
sent2 = "저는 이해하려고 노력하고 있었어요."